#header

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


import nltk 
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
!pip install pyldavis

     |████████████████████████████████| 1.7 MB 9.3 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for pyldavis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136898 sha256=d4a5edd60d94061293cf88c385357c618210d1202659e4bbcd3574c93d8dad1b
  Stored in directory: /root/.cache/pip/wheels/c9/21/f6/17bcf2667e8a68532ba2fbf6d5c72fdf4c7f7d9abfa4852d2f
Successfully built pyldavis


In [ ]:
import pyLDAvis
import pyLDAvis.gensim_models
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import spacy

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable
/usr/local/lib/python3.7/dist-packages/past/builtins/misc.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Mapping
/usr/local/lib/python3.7/dist-packages/scipy/sparse/sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


In [ ]:
from google.colab import auth 
auth.authenticate_user() 
 
import gspread 
from google.auth import default 
creds, _ = default() 
 
gc = gspread.authorize(creds)

In [ ]:
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
stop_words = nltk.corpus.stopwords.words('english')

/usr/local/lib/python3.7/dist-packages/catalogue.py:138: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/usr/local/lib/python3.7/dist-packages/catalogue.py:138: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/usr/local/lib/python3.7/dist-packages/catalogue.py:126: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/usr/local/lib/python3.7/dist-packages/catalogue.py:138: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):


#main

In [ ]:

def process_words(texts, stop_words=stop_words, allowed_tags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    bigram = gensim.models.Phrases(texts, min_count=20, threshold=100)
    trigram = gensim.models.Phrases(bigram[texts], threshold=100)
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    trigram_mod = gensim.models.phrases.Phraser(trigram)
    texts = [[word for word in simple_preprocess(str(doc), deacc=True, min_len=3) if word not in stop_words] for doc in texts]
    texts = [bigram_mod[doc] for doc in texts]
    texts = [trigram_mod[bigram_mod[doc]] for doc in texts]
    
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_tags])
    
    texts_out = [[word for word in simple_preprocess(str(doc), deacc=True, min_len=3) if word not in stop_words] for doc in texts_out]    
    # print(texts_out)
    return texts_out

In [ ]:
def corpus_ready(data):
    data_ready = process_words(data)
    id2word = corpora.Dictionary(data_ready)
    corpus = [id2word.doc2bow(text) for text in data_ready]
    # print(corpus)
    return corpus,id2word

In [ ]:
!pip install --upgrade pandas==1.2
from sklearn import preprocessing

def frequency_counter(data):
    dict_corpus = {}
    dict_={}
    totalfreq=0
    corpus,id2word=corpus_ready(data)
    for i in range(len(corpus)):
        for idx, freq in corpus[i]:
            totalfreq+=freq
            if id2word[idx] in dict_corpus:
                dict_corpus[id2word[idx]] += freq
            else:
                dict_corpus[id2word[idx]] = freq

    dict_['words']=dict_corpus.keys() 
    dict_['freq']= dict_corpus.values()
    dict_df = pd.DataFrame.from_dict(dict_)
    x = dict_df['freq'].values.astype(float).reshape(-1,1) #returns a numpy array
    min_max_scaler = preprocessing.MinMaxScaler()
    x_scaled = min_max_scaler.fit_transform(x)
    dict_df['freqNormalize']=x_scaled
    extension = dict_df[dict_df.freq>50].words.values
    stop_words.extend(extension)
    # print(stop_words)

     |████████████████████████████████| 9.9 MB 8.6 MB/s 
  Attempting uninstall: pandas
    Found existing installation: pandas 1.3.5
    Uninstalling pandas-1.3.5:
      Successfully uninstalled pandas-1.3.5


#generate TM

In [ ]:
def new_corpus(data,test=0):
    if test==0:
        frequency_counter(data)
    data_ready = process_words(data)
    id2word = corpora.Dictionary(data_ready)
    id2word.filter_extremes(no_below=2, no_above=0.5)
    corpus = [id2word.doc2bow(text) for text in data_ready]
    # print(corpus)
    return corpus,id2word

In [ ]:
def topic_model_train(X_train,topics):
    global lda
    corpus,id2word=new_corpus(X_train)
    lda = gensim.models.ldamodel.LdaModel(corpus=corpus, num_topics=topics, id2word=id2word)
    tm_train_results=lda[corpus]
    # print(tm_train_results)
    return tm_train_results

In [ ]:
def topic_prediction_vector(X_test,topics):
    corpus,_ =new_corpus(X_test,1)
    tm_test_results = lda[corpus]
    # print(tm_test_results)
    return tm_test_results

In [ ]:
def create_TM_df(tm_results,topics):
    rows, cols = (len(tm_results), topics)
    feature=[]
    for i in range(rows):
        col = tm_results[i]
        topic_exist=[topic[0] for topic in col]
        for j in range(cols):
            if j in topic_exist:
                continue
            else:
                col.append((j,0))
        col.sort()
        feature.append(col)
    columns = ["Topic-" + str(number) for number in range(1,topics+1)]
    X_TM_df=pd.DataFrame([[y[1] for y in  x] for x in feature],columns=columns)
    return X_TM_df

#Metrics

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

def permanceMetrics(Y, pred, pred_prob):
    acc = accuracy_score(Y,pred)
    pre = precision_score(Y,pred,average='weighted')
    re = recall_score(Y,pred, average='weighted')
    f1 = f1_score(Y,pred, average='weighted')
    acrc = roc_auc_score(Y,pred_prob,multi_class= 'ovr')
    return acc, pre, re, f1, acrc

def avgMetric(met):
    res = np.array(met)
    acc = res[::5].mean()
    pre = res[1::5].mean()
    re = res[2::5].mean()
    f1 = res[3::5].mean()
    acrc = res[4::5].mean()
    return np.array([acc, pre, re, f1, acrc])

#New Version

In [ ]:
import pandas as pd
worksheet1 = gc.open('Final Dataset').get_worksheet(0)
worksheet2 = gc.open('Final Dataset').get_worksheet(1)
rows1 = worksheet1.get_all_values()
rows2 = worksheet2.get_all_values()


stackData1=pd.DataFrame.from_records(rows1,columns=rows1[0])
stackData2=pd.DataFrame.from_records(rows2,columns=rows2[0])


stackData1.drop(0, inplace=True, axis=0)
stackData2.drop(0, inplace=True, axis=0)


stackData=[stackData1, stackData2]
stackData=pd.concat(stackData,ignore_index=True)

stackData['ProcessedBody']=stackData['ProcessedBody']+stackData['Tags']+stackData['Title']

feature=["ProcessedBody", "LOC", "QuestionLength",	"Url+ImageCount",	"Reputation",	"user_badge_bronze_counts",	"user_badge_gold_counts",	"user_badge_silver_counts",	"accept_rate"]
data=stackData[feature].values
y=stackData["Label"].values

print(data.shape, y.shape)

(1245, 9) (1245,)


#Random Forest

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
warnings.filterwarnings("ignore", category=UserWarning) 

from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier as RF
skf = StratifiedKFold(n_splits=10)

i=0
NoOfTopics=36
rm = []
for topic in range(NoOfTopics,NoOfTopics+1):
    print("For total Number of Topics:",topic)
    print('Metrics :  Accuracy \t\tPrecision \t\tRecall \t\tF1-score \tAUC_ROC')
    i=0
    for train_index, test_index in skf.split(data,y):
        X_train_body, X_test_body = data[:,0][train_index], data[:,0][test_index]
        Y_train, Y_test = y[train_index], y[test_index]

        print(X_train_body.shape, X_test_body.shape)
        print(Y_train.shape, Y_test.shape)


        tm_train_results=topic_model_train(X_train_body,topic)
        tm_test_results=topic_prediction_vector(X_test_body,topic)
        X_train_df=create_TM_df(tm_train_results,topic)
        X_test_df=create_TM_df(tm_test_results,topic)


        
        X_train_others,X_test_others=data[:,1:9][train_index], data[:,1:9][test_index]

        X_train_others_df=pd.DataFrame([[x for x in col] for col in X_train_others],columns=feature[1:9])
        X_test_others_df=pd.DataFrame([[x for x in col] for col in X_test_others],columns=feature[1:9])

        X_all_train_df=pd.concat([X_train_df,X_train_others_df], axis=1)
        X_all_test_df=pd.concat([X_test_df,X_test_others_df], axis=1)



        X_train=X_all_train_df.values
        X_test=X_all_test_df.values


        rf = RF(n_estimators = 15, max_depth = 8, criterion='entropy', random_state = 42)



        rf.fit(X_train,Y_train)
        pred = rf.predict(X_test)
        pred_prob = rf.predict_proba(X_test)


        i=i+1
        acc, pre, re, f1, acrc = permanceMetrics(Y_test, pred, pred_prob)
        print('Fold-',i,': ', acc, pre, re, f1, acrc)
        cm = confusion_matrix(Y_test, pred)
        print(len(Y_test))
        print(cm)
        rm += [acc, pre, re, f1]

    acc, pre, re, f1, acrc = avgMetric(rm)
    print('\nAverage: ', acc, pre, re, f1, acrc)

For total Number of Topics: 36
Metrics :  Accuracy 		Precision 		Recall 		F1-score 	AUC_ROC
(1120,) (125,)
(1120,) (125,)
Fold- 1 :  0.592 0.5688205128205128 0.592 0.5104861952861953 0.6099976949690006
125
[[ 1  7  2]
 [ 0 68  6]
 [ 0 36  5]]
(1120,) (125,)
(1120,) (125,)
Fold- 2 :  0.584 0.4436190476190476 0.584 0.48378181818181815 0.6535735822707939
125
[[ 0  3  7]
 [ 0 70  4]
 [ 0 38  3]]
(1120,) (125,)
(1120,) (125,)
Fold- 3 :  0.592 0.4362434782608696 0.592 0.4767731092436975 0.7028985242315664
125
[[ 0  4  6]
 [ 0 72  2]
 [ 0 39  2]]
(1120,) (125,)
(1120,) (125,)
Fold- 4 :  0.656 0.5812391879964696 0.656 0.5932009685230024 0.7297386243100797
125
[[ 0  4  6]
 [ 0 70  4]
 [ 0 29 12]]
(1120,) (125,)
(1120,) (125,)
Fold- 5 :  0.656 0.585939393939394 0.656 0.6022312138728323 0.7574838545271981
125
[[ 0  5  6]
 [ 0 69  5]
 [ 2 25 13]]
(1121,) (124,)
(1121,) (124,)
Fold- 6 :  0.5967741935483871 0.4970814132104454 0.5967741935483871 0.5154591466887588 0.7311577954588534
124
[[ 0  3  7]
 

#XG Boost


In [ ]:
from xgboost import XGBClassifier
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
warnings.filterwarnings("ignore", category=UserWarning) 


kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
i=0;
xg = []
size=20

for train_index, test_index in kfold.split(data,y):
        X_train_body, X_test_body = data[:,0][train_index], data[:,0][test_index]
        y_train, y_test = y[train_index], y[test_index]
        print(X_train_body.shape, X_test_body.shape)
        print(Y_train.shape, Y_test.shape)


        tm_train_results=topic_model_train(X_train_body,topic)
        tm_test_results=topic_prediction_vector(X_test_body,topic)
        X_train_df=create_TM_df(tm_train_results,topic)
        X_test_df=create_TM_df(tm_test_results,topic)


        
        X_train_others,X_test_others=data[:,1:9][train_index], data[:,1:9][test_index]

        X_train_others_df=pd.DataFrame([[x for x in col] for col in X_train_others],columns=feature[1:9])
        X_test_others_df=pd.DataFrame([[x for x in col] for col in X_test_others],columns=feature[1:9])

        X_all_train_df=pd.concat([X_train_df,X_train_others_df], axis=1)
        X_all_test_df=pd.concat([X_test_df,X_test_others_df], axis=1)



        X_train=X_all_train_df.values
        X_test=X_all_test_df.values

        my_model = XGBClassifier(n_estimators=40,learning_rate=0.05, max_depth=8)
        my_model.fit(X_train, y_train)
        pred = my_model.predict(X_test)
        pred_prob = my_model.predict_proba(X_test)
        i=i+1
        acc, pre, re, f1, acrc = permanceMetrics(y_test, pred, pred_prob)
        print('Fold-',i,': ', acc, pre, re, f1, acrc)
        xg += [acc, pre, re, f1, acrc]
        cm = confusion_matrix(y_test, pred)
        print(cm)
acc, pre, re, f1, acrc = avgMetric(xg)
print('\nAverage: ', acc, pre, re, f1, acrc)


(1120,) (125,)
(1121,) (124,)
Fold- 1 :  0.616 0.574538775510204 0.616 0.572464039828665 0.6940823069074581
[[ 3  4  3]
 [ 1 65  8]
 [ 3 29  9]]
(1120,) (125,)
(1121,) (124,)
Fold- 2 :  0.632 0.6113085194375517 0.632 0.6042436674436674 0.6657867183062117
[[ 2  2  6]
 [ 0 63 11]
 [ 1 26 14]]
(1120,) (125,)
(1121,) (124,)
Fold- 3 :  0.648 0.6194195083267248 0.648 0.6073904163393559 0.7142864895425559
[[ 1  4  5]
 [ 1 66  7]
 [ 0 27 14]]
(1120,) (125,)
(1121,) (124,)
Fold- 4 :  0.624 0.5692669683257919 0.624 0.5943924528301887 0.6884031753578257
[[ 0  3  7]
 [ 0 60 14]
 [ 1 22 18]]
(1120,) (125,)
(1121,) (124,)
Fold- 5 :  0.624 0.5473131313131313 0.624 0.5685433526011561 0.6321117965700008
[[ 0  5  6]
 [ 0 67  7]
 [ 2 27 11]]
(1121,) (124,)
(1121,) (124,)
Fold- 6 :  0.6612903225806451 0.6446201362124492 0.6612903225806451 0.6321881307884266 0.6467670885388519
[[ 2  5  3]
 [ 2 64  7]
 [ 0 25 16]]
(1121,) (124,)
(1121,) (124,)
Fold- 7 :  0.5887096774193549 0.5676870158399815 0.5887096774193

#ADA Boost


In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier as RF
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix
# import scikitplot as skplt
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
warnings.filterwarnings("ignore", category=UserWarning) 

kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
i=0;
xg = []
size=20

for train_index, test_index in kfold.split(data,y):
        X_train_body, X_test_body = data[:,0][train_index], data[:,0][test_index]
        y_train, y_test = y[train_index], y[test_index]
        print(X_train_body.shape, X_test_body.shape)
        print(Y_train.shape, Y_test.shape)


        tm_train_results=topic_model_train(X_train_body,topic)
        tm_test_results=topic_prediction_vector(X_test_body,topic)
        X_train_df=create_TM_df(tm_train_results,topic)
        X_test_df=create_TM_df(tm_test_results,topic)


        
        X_train_others,X_test_others=data[:,1:9][train_index], data[:,1:9][test_index]

        X_train_others_df=pd.DataFrame([[x for x in col] for col in X_train_others],columns=feature[1:9])
        X_test_others_df=pd.DataFrame([[x for x in col] for col in X_test_others],columns=feature[1:9])

        X_all_train_df=pd.concat([X_train_df,X_train_others_df], axis=1)
        X_all_test_df=pd.concat([X_test_df,X_test_others_df], axis=1)



        X_train=X_all_train_df.values
        X_test=X_all_test_df.values

        my_model = AdaBoostClassifier(n_estimators=1000,learning_rate=0.05)
        my_model.fit(X_train, y_train)
        pred = my_model.predict(X_test)
        pred_prob = my_model.predict_proba(X_test)
        i=i+1
        acc, pre, re, f1, acrc = permanceMetrics(y_test, pred, pred_prob)
        print('Fold-',i,': ', acc, pre, re, f1, acrc)
        xg += [acc, pre, re, f1, acrc]
        cm = confusion_matrix(y_test, pred)
        print(cm)
acc, pre, re, f1, acrc = avgMetric(xg)
print('\nAverage: ', acc, pre, re, f1, acrc)

(1120,) (125,)
(1121,) (124,)
Fold- 1 :  0.672 0.6018368298368298 0.672 0.6190968855146234 0.5513411467020101
[[ 0  4  6]
 [ 0 69  5]
 [ 0 26 15]]
(1120,) (125,)
(1121,) (124,)
Fold- 2 :  0.624 0.5940989247311828 0.624 0.5893441286441202 0.5884844198396066
[[ 1  4  5]
 [ 0 63 11]
 [ 1 26 14]]
(1120,) (125,)
(1121,) (124,)
Fold- 3 :  0.592 0.521668085106383 0.592 0.5478256203890007 0.46478516862210945
[[ 0  4  6]
 [ 0 62 12]
 [ 1 28 12]]
(1120,) (125,)
(1121,) (124,)
Fold- 4 :  0.64 0.5737142857142856 0.64 0.6015661103979459 0.6572052100578082
[[ 0  3  7]
 [ 0 63 11]
 [ 0 24 17]]
(1120,) (125,)
(1121,) (124,)
Fold- 5 :  0.592 0.5525714285714287 0.592 0.5456744186046514 0.5651939358440906
[[ 1  6  4]
 [ 0 63 11]
 [ 1 29 10]]
(1121,) (124,)
(1121,) (124,)
Fold- 6 :  0.6370967741935484 0.6546015180265655 0.6370967741935484 0.6133765173169012 0.5735303900878442
[[ 1  4  5]
 [ 0 59 14]
 [ 0 22 19]]
(1121,) (124,)
(1121,) (124,)
Fold- 7 :  0.6129032258064516 0.5323071528751754 0.6129032258064

#SVM

In [ ]:
from sklearn import svm
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix
# import scikitplot as skplt
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
warnings.filterwarnings("ignore", category=UserWarning) 

kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
i=0;
xg = []
size=20

for train_index, test_index in kfold.split(data,y):
        X_train_body, X_test_body = data[:,0][train_index], data[:,0][test_index]
        y_train, y_test = y[train_index], y[test_index]
        print(X_train_body.shape, X_test_body.shape)
        print(Y_train.shape, Y_test.shape)


        tm_train_results=topic_model_train(X_train_body,topic)
        tm_test_results=topic_prediction_vector(X_test_body,topic)
        X_train_df=create_TM_df(tm_train_results,topic)
        X_test_df=create_TM_df(tm_test_results,topic)


        
        X_train_others,X_test_others=data[:,1:9][train_index], data[:,1:9][test_index]

        X_train_others_df=pd.DataFrame([[x for x in col] for col in X_train_others],columns=feature[1:9])
        X_test_others_df=pd.DataFrame([[x for x in col] for col in X_test_others],columns=feature[1:9])

        X_all_train_df=pd.concat([X_train_df,X_train_others_df], axis=1)
        X_all_test_df=pd.concat([X_test_df,X_test_others_df], axis=1)



        X_train=X_all_train_df.values
        X_test=X_all_test_df.values

        my_model = svm.SVC(decision_function_shape='ovr',probability=True)
        my_model.fit(X_train, y_train)
        pred = my_model.predict(X_test)
        pred_prob = my_model.predict_proba(X_test)
        i=i+1
        acc, pre, re, f1, acrc = permanceMetrics(y_test, pred, pred_prob)
        print('Fold-',i,': ', acc, pre, re, f1, acrc)
        xg += [acc, pre, re, f1, acrc]
        cm = confusion_matrix(y_test, pred)
        print(cm)
acc, pre, re, f1, acrc = avgMetric(xg)
print('\nAverage: ', acc, pre, re, f1, acrc)